In [4]:
import numpy as np
import pandas as pd

In [28]:
data = pd.read_csv(r'C:\Users\Shkolaynski.MOKB\DataspellProjects\ITMO_DL_in_practice_02\Source\train.csv', index_col='pair_id')
data.head()

,name_1,name_2,is_duplicate
pair_id,,,
1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
4,Powermax Rubber Factory,Co. One,0
5,Tress A/S,Longyou Industries Park Zhejiang,0


In [67]:
df = pd.concat([data.name_1, data.name_2]).drop_duplicates().reset_index(drop=True)
df = pd.DataFrame(data=df, columns=['name'])
df.tail()

,name
18017,Plastic Packaging (Pty) Ltd.
18018,'Hengshui Mechanical & Electrical Building Co....
18019,"Jiangsu Baoli International Investment Co., Lt..."
18020,Lanxess AG
18021,"TGI IMPORT EXPORT CO., LTD."


In [59]:
# найдём строки ближайшие к данной
input_str = df.sample(1, random_state=42).iloc[0]
input_str = 'Просто левая строка для проверки'
input_str

'Просто левая строка для проверки'

In [106]:
import re
import unicodedata
from cleanco import basename
from transliterate.decorators import transliterate_function


def preprocessing(x: str) -> str:
    
    def rus_preprocess(x: str) -> str:
        forms_of_ownership = [
            'ооо',
            'оао',
            'общество с ограниченной ответственностью',
            'открытое акционерное общество',
            'филиал компании'
        ]
        
        for form in forms_of_ownership:
            pattern = re.compile(form)
            x = pattern.sub('', x)
            
        return x
    
    @transliterate_function(language_code='ru', reversed=True)
    def translit(x: str) -> str:
        return x
    
    
    x = re.sub(r'[^\w\s]+|[\d]+',' ', x)
    x = x.strip().lower()
    x = basename(x) if not bool(set(rus_alphabet).intersection(set(x.lower()))) else rus_preprocess(x)
    x = unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode() if not bool(set(rus_alphabet).intersection(set(x.lower()))) else translit(x)
    x = basename(x)
    
    return ' '.join([s for s in x.split()]) if len(x.split()) != 0 else '-' * 5
    

a = ord('а') # кирилическая "А"
rus_alphabet = ''.join([chr(i) for i in range(a,a+6)] + [chr(a+33)] + [chr(i) for i in range(a+6,a+32)])
    
input_str_alt = preprocessing(input_str)
test_df = df.copy()
test_df['name_alt'] = test_df.name.apply(lambda x: preprocessing(x))
print(input_str, ' | ', input_str_alt)
test_df.tail()

Просто левая строка для проверки  |  prosto levaja stroka dlja proverki


,name,name_alt
18017,Plastic Packaging (Pty) Ltd.,plastic packaging
18018,'Hengshui Mechanical & Electrical Building Co....,hengshui mechanical electrical building
18019,"Jiangsu Baoli International Investment Co., Lt...",jiangsu baoli international investment co ltd ...
18020,Lanxess AG,lanxess
18021,"TGI IMPORT EXPORT CO., LTD.",tgi import export


In [107]:
from fuzzywuzzy import fuzz
import jellyfish

import warnings

warnings.filterwarnings("ignore")


# Token Set Ratio
test_df.insert(1, 'tsr', test_df.name_alt.apply(lambda x: fuzz.token_set_ratio(input_str_alt, x) / 100))

# Jaro-Winkler Similarity
test_df.insert(2, 'jws', test_df.name_alt.apply(lambda x: jellyfish.jaro_winkler_similarity(input_str_alt, x)))

test_df.to_csv(r'C:\Users\Shkolaynski.MOKB\DataspellProjects\ITMO_DL_in_practice_02\Source\eval.csv')

In [117]:
print(input_str, ' | ', input_str_alt, '\n')
res = test_df.loc[test_df.sort_values(['tsr', 'jws'], ascending=False).index, ['name', 'tsr', 'jws']].head()
if res.iloc[0, -2:][['tsr', 'jws']].mean() > 0.5:
    print(res)
else:
    print('Строк похожих на заданную не обнаружено')

Просто левая строка для проверки  |  prosto levaja stroka dlja proverki 

                                     name   tsr       jws
5832             Poder Logistico Sa De Cv  0.55  0.639998
5695           Productos Roche Panama S A  0.53  0.675478
6813            Etam Pret A Porter S.A.S.  0.52  0.660690
12961            Pt Penta Jaya Laju Motor  0.52  0.648693
1961   Saro Rubber & Allied Products Ltd.  0.52  0.631369
